In [1]:
import pandas as pd
import numpy as np
import math

ratings = pd.read_csv('ratings.csv')
movies = pd.read_csv('movies.csv')

movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [2]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [3]:
m = len(movies)
tt = movies.genres

genres_list = []
item_prof = []
for i in range(0, m):
    aa = tt.iloc[i].split('|')
    genres_list += aa
    item_prof.append(aa)

In [4]:
genres_list[0:9]

['Adventure',
 'Animation',
 'Children',
 'Comedy',
 'Fantasy',
 'Adventure',
 'Children',
 'Fantasy',
 'Comedy']

In [5]:
item_prof[0:4]

[['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy'],
 ['Adventure', 'Children', 'Fantasy'],
 ['Comedy', 'Romance'],
 ['Comedy', 'Drama', 'Romance']]

In [6]:
from mlxtend.preprocessing import TransactionEncoder

te = TransactionEncoder()
te_arr = te.fit_transform(item_prof)
te_arr = te_arr.astype('int')
df = pd.DataFrame(te_arr, columns=te.columns_)
df.head()

,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0
4,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [7]:
movieID = movies.movieId[0:m]
df.index = movieID
df.head()

,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movieId,,,,,,,,,,,,,,,,,,,,
1,0,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0
5,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [8]:
ratings_mean_cnt = pd.DataFrame(ratings.groupby('movieId')['rating'].mean())
ratings_mean_cnt

,rating
movieId,
1,3.920930
2,3.431818
3,3.259615
4,2.357143
5,3.071429
...,...
193581,4.000000
193583,3.500000
193585,3.500000


In [9]:
user_movie = [["Drama", "Thriller"], ["Adventure", "Comedy", "Crime", "Romance"], ["Comedy"], ["Comedy"]]
user_rating = [6, 10, 8, 8]

te = TransactionEncoder()
te_arr = te.fit_transform(user_movie)
te_arr = te_arr.astype('int')
user_df = pd.DataFrame(te_arr, columns=te.columns_)
user_df

,Adventure,Comedy,Crime,Drama,Romance,Thriller
0,0,0,0,1,0,1
1,1,1,1,0,1,0
2,0,1,0,0,0,0
3,0,1,0,0,0,0


In [10]:
for i in range(4):
    user_df.iloc[i] *= user_rating[i]

user_df

,Adventure,Comedy,Crime,Drama,Romance,Thriller
0,0,0,0,6,0,6
1,10,10,10,0,10,0
2,0,8,0,0,0,0
3,0,8,0,0,0,0


In [11]:
features = (user_df/2).mean()
features

Adventure    1.25
Comedy       3.25
Crime        1.25
Drama        0.75
Romance      1.25
Thriller     0.75
dtype: float64

In [12]:
df = df[user_df.columns]
df

,Adventure,Comedy,Crime,Drama,Romance,Thriller
movieId,,,,,,
1,1,1,0,0,0,0
2,1,0,0,0,0,0
3,0,1,0,0,1,0
4,0,1,0,1,1,0
5,0,1,0,0,0,0
...,...,...,...,...,...,...
193581,0,1,0,0,0,0
193583,0,1,0,0,0,0
193585,0,0,0,1,0,0


In [13]:
dic = {}
for idx in ratings_mean_cnt.index:
    dic[idx] = True

In [14]:
movieId_NOT_rated = []
for idx in df.index:
    if idx not in dic:
        movieId_NOT_rated.append(idx)
movieId_NOT_rated

[1076,
 2939,
 3338,
 3456,
 4194,
 5721,
 6668,
 6849,
 7020,
 7792,
 8765,
 25855,
 26085,
 30892,
 32160,
 32371,
 34482,
 85565]

In [15]:
df = df.drop(movieId_NOT_rated)
df

,Adventure,Comedy,Crime,Drama,Romance,Thriller
movieId,,,,,,
1,1,1,0,0,0,0
2,1,0,0,0,0,0
3,0,1,0,0,1,0
4,0,1,0,1,1,0
5,0,1,0,0,0,0
...,...,...,...,...,...,...
193581,0,1,0,0,0,0
193583,0,1,0,0,0,0
193585,0,0,0,1,0,0


In [16]:
for i in range(len(df)):
    df.iloc[i] *= ratings_mean_cnt.iloc[i].rating

df

,Adventure,Comedy,Crime,Drama,Romance,Thriller
movieId,,,,,,
1,3.920930,3.920930,0.0,0.000000,0.000000,0.0
2,3.431818,0.000000,0.0,0.000000,0.000000,0.0
3,0.000000,3.259615,0.0,0.000000,3.259615,0.0
4,0.000000,2.357143,0.0,2.357143,2.357143,0.0
5,0.000000,3.071429,0.0,0.000000,0.000000,0.0
...,...,...,...,...,...,...
193581,0.000000,4.000000,0.0,0.000000,0.000000,0.0
193583,0.000000,3.500000,0.0,0.000000,0.000000,0.0
193585,0.000000,0.000000,0.0,3.500000,0.000000,0.0


In [17]:
from sklearn.metrics.pairwise import cosine_similarity

lst = cosine_similarity([features], df.iloc[:])[0]
lst

array([0.78633365, 0.30890103, 0.78633365, ..., 0.18534062, 0.        ,
       0.80314268])

In [18]:
arr = []
for i in range(len(lst)):
    arr.append((lst[i], i))

arr = sorted(arr)[::-1]
arr[:5]

[(0.8649228885013016, 7016),
 (0.8649228885013016, 89),
 (0.8564973941930834, 5760),
 (0.8564973941930832, 743),
 (0.8203828312573851, 7163)]

In [19]:
_arr = []
for x in arr[:5]:
    _arr.append(x[1])
_arr

[7016, 89, 5760, 743, 7163]

In [20]:
# for idx in _arr:
#     print(_df.iloc[idx])

In [21]:
print("Top 5 suggestions:")
for i in range(m):
    if movies.iloc[i].name in _arr:
        print('\t', movies.iloc[i].title)

Top 5 suggestions:
	 Bottle Rocket (1996)
	 Beat the Devil (1953)
	 Imaginary Heroes (2004)
	 X-Men Origins: Wolverine (2009)
	 New York, I Love You (2009)
